# Comparing a USGSCSM and ISIS camera for Kaguya Terrain Camera

In [1]:
import pvl
import os
import tempfile
import csmapi
import json

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
import knoten
import ale
from knoten import vis
from ale.drivers.kaguya_drivers import KaguyaTcPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
from ale import util
from IPython.display import Image
from pysis import isis
from pysis.exceptions import ProcessError

import plotly
plotly.offline.init_notebook_mode(connected=True)

ModuleNotFoundError: No module named 'ale'

## Make a CSM sensor model
Requires TC1S2B0_01_02842S506E1942.img in data directory

In [ ]:
fileName = 'data/TC1S2B0_01_02842S506E1942.img'
camera = knoten.csm.create_csm(fileName)

## Ingest the image and spiceinit

In [ ]:
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.kaguyatc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')#, iak='/home/arsanders/testData/kaguyaTcAddendum007.ti')
except ProcessError as e:
    print(e.stderr)

img_label = pvl.load(cub_loc)
year = img_label['IsisCube']['Instrument']['StartTime'].year
with tempfile.NamedTemporaryFile("r+") as metakernel:
    metakernel_path = os.path.split(metakernel.name)[0]
    metakernel.name = metakernel_path + '/kaguya{}.tm'.format(year)
    util.write_metakernel_from_cube(cub_loc, metakernel.name)

    ale.config['kaguya'] = metakernel_path
    with KaguyaTcPds3NaifSpiceDriver(fileName) as driver:
        usgscsmString = to_usgscsm(driver)
        usgscsm_dict = json.loads(usgscsmString)

        csm_isd = os.path.splitext(fileName)[0] + '.json'
        json.dump(usgscsm_dict, open(csm_isd, 'w'))

## Compare USGS CSM and ISIS pixels

In [ ]:
csmisis_diff_lv_plot, csmisis_diff_ephem_plot, external_orientation_data = vis.external_orientation_diff(csm_isd, cub_loc, 10, 50, 600, 600)

In [ ]:
csmisis_diff_lv_plot

In [ ]:
csmisis_diff_ephem_plot

In [ ]:
csmisis_diff_lv_plot_bytes = csmisis_diff_lv_plot.to_image(format="png")
csmisis_diff_ephem_plot_bytes = csmisis_diff_ephem_plot.to_image(format="png")
Image(csmisis_diff_lv_plot_bytes)

In [ ]:
Image(csmisis_diff_ephem_plot_bytes)

In [ ]:
external_orientation_data[['diffx', 'diffy', 'diffz', 'diffu', 'diffv', 'diffw']].describe()

In [ ]:
isis2csm_plot, csm2isis_plot, isiscsm_plotlatlon, isiscsm_plotbf, isis2csm_data, csm2isis_data, isiscsm_latlondata, isiscsm_bfdata = vis.reprojection_diff(csm_isd, cub_loc, 10, 50, 500, 500)

In [ ]:
Image(isis2csm_plot.to_image())

In [ ]:
isis2csm_data[['diff line', 'diff sample']].describe()

In [ ]:
Image(csm2isis_plot.to_image())

In [ ]:
csm2isis_data[['diff line', 'diff sample']].describe()

In [ ]:
Image(isiscsm_plotlatlon.to_image())

In [ ]:
Image(isiscsm_plotbf.to_image())

In [ ]:
isiscsm_bfdata[['diffx', 'diffy', 'diffz']].describe()